In [1]:
# Import Statemnts
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import joblib


In [2]:
# Change these values to change the time period
Month = 1 
Hour = 1

# Read in the data
df = pd.read_csv("generator_level_lbmp.csv")

df = df[(df["Month"] == Month) & (df["Hour"] == Hour)].reset_index(drop=True)
df = pd.get_dummies(df, prefix="Zone", columns=['Name'])

df.head()

,Unnamed: 0,Month,Hour,Average LBMP,generator_name,summer_capacity,winter_capacity,power_source,Zone,renewable,...,Zone_CENTRL,Zone_DUNWOD,Zone_GENESE,Zone_HUD VL,Zone_LONGIL,Zone_MHK VL,Zone_MILLWD,Zone_N.Y.C.,Zone_NORTH,Zone_WEST
0,0,1,1,32.722258,Allens Falls,4.4,4.4,Water,F,1,...,False,False,False,False,False,False,False,False,False,False
1,1,1,1,32.722258,"Athens 1, 2, and 3",994.1,1202.8,Natural Gas,F,0,...,False,False,False,False,False,False,False,False,False,False
2,2,1,1,32.722258,Baldwinsville 1 and 2,0.6,0.6,Water,F,1,...,False,False,False,False,False,False,False,False,False,False
3,3,1,1,32.722258,Beacon LESR,0.0,0.0,Flywheel,F,0,...,False,False,False,False,False,False,False,False,False,False
4,4,1,1,32.722258,Beardslee 1 and 2,20.0,20.0,Water,F,1,...,False,False,False,False,False,False,False,False,False,False


In [3]:
# Predict Demand for Day Ahead and Hour Ahead 
# Use the following features:
#   - Hour of Day
#   - Month of Year
#   - Zone


ALL_ZONES = ["Zone_CAPITL", "Zone_CENTRL", "Zone_DUNWOD", "Zone_GENESE", "Zone_HUD VL", "Zone_LONGIL", "Zone_MHK VL", "Zone_MILLWD", "Zone_N.Y.C.", "Zone_NORTH", "Zone_WEST"]
NUM_GENERATORS = len(df)
print(NUM_GENERATORS)

model=joblib.load("linear_predictor")
TOTAL_DEMAND = 0
Demand_per_zone = {}
if Month <= 10 and Month >= 5: 
    df["max_cap"] = df["summer_capacity"]
else: 
    df["max_cap"] = df["winter_capacity"]


for zone in ALL_ZONES:
    new_data = pd.DataFrame({"Month": Month, "Hour": Hour, "Zone_CAPITL": [zone=="Zone_CAPITL"], "Zone_CENTRL": [zone=="Zone_CENTRL"], "Zone_DUNWOD": [zone=="Zone_DUNWOD"], "Zone_GENESE": [zone=="Zone_GENESE"], "Zone_HUD VL": [zone=="Zone_HUD VL"], "Zone_LONGIL": [zone=="Zone_LONGIL"], "Zone_MHK VL": [zone=="Zone_MHK VL"], "Zone_MILLWD": [zone=="Zone_MILLWD"], "Zone_N.Y.C.": [zone=="Zone_N.Y.C."], "Zone_NORTH": [zone=="Zone_NORTH"], "Zone_WEST": [zone=="Zone_WEST"]})
    TOTAL_DEMAND += model.predict(new_data)[0]
    Demand_per_zone[zone] = model.predict(new_data)[0]
print("Total Demand:", TOTAL_DEMAND)
print("Demand per zone:", Demand_per_zone)


371
Total Demand: 14470.329567122506
Demand per zone: {'Zone_CAPITL': 1061.3218676806284, 'Zone_CENTRL': 1456.7613376207414, 'Zone_DUNWOD': 404.84036652807276, 'Zone_GENESE': 833.5540071892235, 'Zone_HUD VL': 816.0764868778075, 'Zone_LONGIL': 1984.1089891114143, 'Zone_MHK VL': 588.170969635473, 'Zone_MILLWD': 88.80335856525403, 'Zone_N.Y.C.': 5349.283641720214, 'Zone_NORTH': 435.2293347540227, 'Zone_WEST': 1452.1792074396528}


In [4]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [5]:
### Decision Variables ###
X = m.addVars((i for i in range(0, NUM_GENERATORS)), vtype=gp.GRB.BINARY, name='X')

In [6]:
# Total Supply >= Demand Contraint
m.addConstr(sum(X[i] * df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= TOTAL_DEMAND, name = "Total Supply >= Demand")

# Renewable Energy Contraint 
# m.addConstr(sum(X[i] * df["renewable"][i] * df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= 0.7 * TOTAL_DEMAND, name = "Renewable Energy Supply")

# Regional Energy Production Contraint 
# for zone in ALL_ZONES:
#    m.addConstr(sum(X[i] * df[zone][i] * df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= 0.5 * Demand_per_zone[zone], name = f"Regional Energy Supply {zone}")



<gurobi.Constr *Awaiting Model Update*>

In [7]:
# Objective Function: Minimize Cost
m.setObjective(sum(df["Average LBMP"][i] * df["max_cap"][i] * X[i] for i in range(0, NUM_GENERATORS)),GRB.MINIMIZE)


In [8]:
m.update() # Update model parameters
m.write("OptimalSupply.lp") # Write model to file
# Solve
m.optimize()


In [9]:
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))


Objective value:  366435.57
